# Import Table 1 from ukb22598.html


In [2]:
import os
#import requests 
import pandas as pd
import bs4
from bs4 import BeautifulSoup
#from lxml import html
from html_table_extractor.extractor import Extractor
import numpy as np

## Parse Table 1 from html format into csv format using local html file

In [3]:
# setup working directory
os.chdir("/projects/ps-janssen3/dsci-pa/yhuan162/temp_project/ukbb/data")

### path for load ukb22598.html
path="/projects/ps-janssen3/dsci-pa/yhuan162/temp_project/ukbb/ukb22598.html"

f = open(path)      # open html file
#soup = BeautifulSoup(f,"lxml")
soup = bs4.BeautifulSoup(f, 'html.parser')
f.close()

extractor = Extractor(soup)
extractor.parse()
#extractor.return_list()[2:5]   #table 1 starts at row 2
#extractor.return_list()[8777:8780]   #table 1 ends at row 8780
col_labels=extractor.return_list()[2]   #colnames for table 1 

tab1=extractor.return_list()[3:8780] 
df_tab1=pd.DataFrame(tab1[0:],columns=col_labels)
df_tab1.to_csv('ukb22598_html_tab1.csv',index=False)
df_tab1.head()




,Column,UDI,Count,Type,Description
0,0,eid,502602,Sequence,Encoded anonymised participant ID
1,1,21-0.0,500887,Categorical (single),Weight methodUses data-coding 100261 comprises...
2,2,21-1.0,20336,Categorical (single),Weight methodUses data-coding 100261 comprises...
3,3,21-2.0,23167,Categorical (single),Weight methodUses data-coding 100261 comprises...
4,4,31-0.0,502602,Categorical (single),SexUses data-coding 9 comprises 2 Integer-valu...


In [4]:
### Add variable tier infomation

tier_path='/projects/ps-janssen3/dsci-pa/yhuan162/temp_project/ukbb/load_Variable_Triage.csv'
tier_table=pd.read_csv(tier_path)
tier_table.head()
tier_table_for_merge=tier_table.drop(columns='Column')
tier_table_for_merge.head()

,UDI,Tier
0,eid,1
1,21-0.0,1
2,21-1.0,1
3,21-2.0,1
4,31-0.0,1


In [5]:
df_tab1_tier=pd.merge(df_tab1,tier_table_for_merge, on='UDI', how='left')

In [6]:
df_tab1_tier.head()

,Column,UDI,Count,Type,Description,Tier
0,0,eid,502602,Sequence,Encoded anonymised participant ID,1
1,1,21-0.0,500887,Categorical (single),Weight methodUses data-coding 100261 comprises...,1
2,2,21-1.0,20336,Categorical (single),Weight methodUses data-coding 100261 comprises...,1
3,3,21-2.0,23167,Categorical (single),Weight methodUses data-coding 100261 comprises...,1
4,4,31-0.0,502602,Categorical (single),SexUses data-coding 9 comprises 2 Integer-valu...,1


In [9]:
### Extract data coding infomation as variable DC
### Extract categorical levels infomation as variable CatLv

df_tab1ex=df_tab1_tier
df_des=df_tab1ex[['Description']]
#print(df_des)
df_des.head()

pdf=df_des.Description.str.partition('Uses data-coding ')

pdf.head()
ppdf=pdf[[2]]
ppdf.columns=['dc']
pppdf=ppdf.dc.str.partition(' ')
dccol=pppdf[[0]]
dccol.columns=['data_coding']


catcol=df_des.Description.str.partition('comprises ')

catcol.head()
catcol2=catcol[[2]]
catcol2.columns=['cat']
catcol3=catcol2.cat.str.partition(' ')
catcol3
catcol4=catcol3[[0]]
catcol4.columns=['CatLv']


df_tab1ex['DC']=dccol
df_tab1ex['CatLv']=catcol4
df_tab1ex.CatLv.describe()

df_tab1ex.head()


,Column,UDI,Count,Type,Description,Tier,DC,CatLv
0,0,eid,502602,Sequence,Encoded anonymised participant ID,1,,
1,1,21-0.0,500887,Categorical (single),Weight methodUses data-coding 100261 comprises...,1,100261,5
2,2,21-1.0,20336,Categorical (single),Weight methodUses data-coding 100261 comprises...,1,100261,5
3,3,21-2.0,23167,Categorical (single),Weight methodUses data-coding 100261 comprises...,1,100261,5
4,4,31-0.0,502602,Categorical (single),SexUses data-coding 9 comprises 2 Integer-valu...,1,9,2


In [11]:
dfchk=df_tab1ex.copy()
dfchk.head()

,Column,UDI,Count,Type,Description,Tier,DC,CatLv
0,0,eid,502602,Sequence,Encoded anonymised participant ID,1,,
1,1,21-0.0,500887,Categorical (single),Weight methodUses data-coding 100261 comprises...,1,100261,5
2,2,21-1.0,20336,Categorical (single),Weight methodUses data-coding 100261 comprises...,1,100261,5
3,3,21-2.0,23167,Categorical (single),Weight methodUses data-coding 100261 comprises...,1,100261,5
4,4,31-0.0,502602,Categorical (single),SexUses data-coding 9 comprises 2 Integer-valu...,1,9,2


In [12]:
### break UDI (#####-m.n) into two parts: 
### filed id (######) as udip1 
### m.n as udip2


df_udi_List = df_tab1['UDI'].tolist()
udi_long=[i.split('-', 1)[0] for i in df_udi_List]
udi_long2=[i.split('-', 1)[-1] for i in df_udi_List]
print(len(udi_long),len(udi_long2),len(df_udi_List),len(dfchk))
#udi_long2
#udi_long[0]=''
#udi_long2[0]=''

dfchk['udip1']=udi_long
dfchk['udip2']=udi_long2
#dfchk0=dfchk

### break udip2 into two parts: 
### m.n --> m as udip2m 
###         n as udip2n


df_udip2_List = dfchk['udip2'].tolist()
udip2_m=[i.split('.', 1)[0] for i in df_udip2_List]
udip2_n=[i.split('.', 1)[-1] for i in df_udip2_List]
udip2_m[0]=-999
udip2_n[0]=-999

udip2_m = [int(i) for i in udip2_m]
udip2_n = [int(i) for i in udip2_n]

dfchk['udip2m']=udip2_m
dfchk['udip2n']=udip2_n

8777 8777 8777 8777


In [14]:
# add observed / missing count / pct
dfred_mischk=dfchk['Count']
len(dfred_mischk)
type(dfred_mischk)
#dfred_mischk
#dfred_mischk.loc.Column=dfred_t0r0.Column.astype(int)
dfred_mischk_col=dfred_mischk.astype(int)
type(dfred_mischk_col)
dfchk['miss_ct']=502602-dfred_mischk_col
dfchk['miss_pt']=(502602-dfred_mischk_col)/502602
dfchk['obs_ct']=dfred_mischk_col
dfchk['obs_pt']=dfred_mischk_col/502602


In [16]:
### save impoved table1

dfchk.to_csv('ukb22598_html_tab1_adv.csv',index=None)
dfchk.head()


,Column,UDI,Count,Type,Description,Tier,DC,CatLv,udip1,udip2,udip2m,udip2n,miss_ct,miss_pt,obs_ct,obs_pt
0,0,eid,502602,Sequence,Encoded anonymised participant ID,1,,,eid,eid,-999,-999,0,0.000000,502602,1.000000
1,1,21-0.0,500887,Categorical (single),Weight methodUses data-coding 100261 comprises...,1,100261,5,21,0.0,0,0,1715,0.003412,500887,0.996588
2,2,21-1.0,20336,Categorical (single),Weight methodUses data-coding 100261 comprises...,1,100261,5,21,1.0,1,0,482266,0.959539,20336,0.040461
3,3,21-2.0,23167,Categorical (single),Weight methodUses data-coding 100261 comprises...,1,100261,5,21,2.0,2,0,479435,0.953906,23167,0.046094
4,4,31-0.0,502602,Categorical (single),SexUses data-coding 9 comprises 2 Integer-valu...,1,9,2,31,0.0,0,0,0,0.000000,502602,1.000000
